In [ ]:
import pickle
import concurrent.futures
import itertools
import time

from bikewaysim.paths import config
from bikewaysim.impedance_calibration import stochastic_optimization
from bikewaysim.general_utils import print_elapsed_time

#NOTE relative import, do not move this file from the directory
from step_1_calibration_experiments import all_calibrations

'''
Runs the validation first
'''

start_time = time.time()

NUM_RUNS = 1 # Number of times to run each calibration
MAX_WORKERS = 14 # For my machine this takes about ~70% CPU and ~80% Memory

print('Found these calibration settings:')
print([x['calibration_name'] for x in all_calibrations])
print('Running each calibration',NUM_RUNS,'times')

# NOTE comment out lines 41-53 to run calibration on all matched traces
# Import subsets
with (config['calibration_fp']/'validation/training_folds.pkl').open('rb') as fh:
    training_folds = pickle.load(fh)

with (config['calibration_fp']/'validation/bootstrap_samples.pkl').open('rb') as fh:
    bootstrap_samples = pickle.load(fh)

# # select which subsets to calibrate
# subset_ids = ['random'] # or uuserid in string format
# subsets = [x for x in subsets if x[0] in subset_ids]
# print([x[0] for x in subsets])

all_calibrations = [x for x in all_calibrations if x['calibration_name'] == 'validation']

# Add users to the calibration list
kfold_calibrations = [{**calibration_dict,**{'subset':subset}} for subset, calibration_dict in itertools.product(training_folds,all_calibrations)]
bootstrap_calibrations = [{**calibration_dict,**{'subset':subset}} for subset, calibration_dict in itertools.product(bootstrap_samples,all_calibrations)]

# validation runs will run first and then bootstrap runs
all_calibrations = kfold_calibrations + bootstrap_calibrations

# END COMMENT BLOCK

# Create a list of (script, run_num) pairs for NUM_RUNS
tasks = [(calibration_dict, run_num, NUM_RUNS) for calibration_dict in all_calibrations for run_num in range(NUM_RUNS)]

# Sort tasks by the run number, so that one full run completes first
tasks = sorted(tasks,key=lambda x: x[1])

In [ ]:
all_calibrations[0]['subset'][0]